In [2]:
import grpc
import tensorflow as tf 
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [4]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [5]:
from keras_image_helper import create_preprocessor

In [6]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [7]:
url = 'http://bit.ly/mlbookcamp-pants'
x = preprocessor.from_url(url)

In [8]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [9]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"

pb_request.inputs["input_8"].CopyFrom(np_to_protobuf(x))

In [11]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [12]:
pb_response

model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}
outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.87986457
    float_val: -4.75631189
    float_val: -2.35953331
    float_val: -1.08926427
    float_val: 9.90378475
    float_val: -2.82618117
    float_val: -3.64831042
    float_val: 3.24115634
    float_val: -2.61209583
    float_val: -4.852036
  }
}

In [13]:
preds = pb_response.outputs["dense_7"].float_val

In [14]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595333099365234,
 'outwear': -1.0892642736434937,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.648310422897339,
 'shorts': 3.2411563396453857,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}